# A first mini-example for the OpenAI API

In [3]:
!pip show openai

Name: openai
Version: 2.6.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /home/juebrauer/miniconda3/envs/env_teaching/lib/python3.12/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [4]:
f = open("my_secrete_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()
print(my_secrete_key[:5])

sk-pr


In [9]:
import os
from openai import OpenAI

client = OpenAI(api_key=my_secrete_key)

response = client.responses.create(
    model="gpt-5",
    instructions="",
    input="Was für Informatik-Masterstudiengänge hat die Hochschule Kempten?",
)

print(response.output_text)

Kurzfassung:
- Informatik (M.Sc.), Vollzeit, 3 Semester, Fakultät Informatik

Hinweis: Die Hochschule Kempten bietet zusätzlich immer wieder berufsbegleitende Weiterbildungs‑Master mit IT-Bezug über die Graduate School an; das konkrete Angebot kann sich ändern.

Möchtest du nur die regulären Vollzeit-Master der Fakultät Informatik, oder auch die berufsbegleitenden/Weiterbildungsprogramme? Wenn du magst, nenne ich dir Inhalte, Zulassung und Fristen und zeige dir, wo du die aktuellen Infos auf der Website findest.


# My first chat-bot

In [ ]:
import os
from openai import OpenAI

f = open("my_secret_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()

client = OpenAI(api_key=my_secrete_key)


history = []
while True:

    user_input = input("Du> ")
    if user_input == "exit":
        break

    history.append( { "role": "user", "content": user_input} )

    response = client.responses.create(
        model="gpt-5-mini",
        instructions="",
        input=history
    )

    history.append( { "role": "assistant", "content": response.output_text} )

    print(f"Du> {user_input}\n")
    print(f"Bot> {response.output_text}\n")

print("history:")
print("-"*60)
print(history)

Du> Was für Informatik-Masterstudiengänge hat die Hochschule Kempten aktuell im Wintersemester 2025/2026 anzubieten?

Bot> Ich kann nicht live ins Internet schauen, daher habe ich keinen Zugriff auf die aktuelle Studiengangsliste für das Wintersemester 2025/2026. Ich kann Ihnen aber helfen, die Information schnell selbst zu finden oder eine Anfrage an die Hochschule zu formulieren. Vorschläge:

1) Schnell selber nachsehen (Schritte)
- Webseite: https://www.hs‑kempten.de
- Menü: „Studium“ → „Studienangebot“ oder „Masterstudiengänge“ bzw. die Fakultät „Informatik / Wirtschaftsinformatik“ anklicken.
- Achten Sie auf Hinweise „Studienbeginn Wintersemester 2025/2026“, Zulassungsart (konsekutiv/nicht-konsekutiv) und Studienbeginn (WS).
- Alternativ Suchbegriffe in einer Suchmaschine: „Hochschule Kempten Master Informatik“, „HS Kempten Masterstudiengänge 2025/2026“, „Hochschule Kempten Master IT Sicherheit“, „Hochschule Kempten Master Data Science“.

2) Was ich für Sie tun kann (wenn Sie möch

In [1]:
!pip install weatherincity

In [ ]:
import weatherincity

def get_current_temperature(cityname):
    weather_str = weatherincity.get_current_temperature(cityname)
    return weather_str

# Function calling (Tool calling)

In [2]:
from openai import OpenAI
import json

f = open("my_secrete_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()
client = OpenAI(api_key=my_secrete_key)

# 1. Define a list of callable tools for the model
tools_list = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
history = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools_list,
    input=history,
)

# Save function call outputs for subsequent requests
history += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            history.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(history)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools_list,
    input=history,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_00b0857374b111bd006900d12bbba48193b56afaad5ec6e773', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_22h2sbooZYTCRvwhualNPKS2', name='get_horoscope', type='function_call', id='fc_00b0857374b111bd006900d12ce7d08193901678b88d178d04', status='completed'), {'type': 'function_call_output', 'call_id': 'call_22h2sbooZYTCRvwhualNPKS2', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_00b0857374b111bd006900d12df80081938a7f275cacdd0619",
  "created_at": 1761661230.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_00b08

# Function calling more detailed

In [65]:
from openai import OpenAI
import json

f = open("my_secrete_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()
client = OpenAI(api_key=my_secrete_key)

tools_list = [
    {
        "type": "function",
        "name": "weather_in_city",
        "description": "Returns the weather in a given city with temperature information specified in Fahrenheit",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city for which to retrieve weather information",
                },
            },
            "required": ["city"],
        },
    },
    {
        "type": "function",
        "name": "fahrenheit_to_celsius",
        "description": "Converts a temperature given in Fahrenheit to Celsius",
        "parameters": {
            "type": "object",
            "properties": {
                "fahrenheit": {
                    "type": "number",
                    "description": "Temperature in Fahrenheit",
                },
            },
            "required": ["fahrenheit"],
        },
    },
]

def weather_in_city(city):
    return f"The weather in {city} is sunny and warm (45°F)"

def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit-32)*5/9
    return celsius


# Create a running input list we will add to over time
history = [
    {"role": "user", "content": "What is the current weather in Kempten?"}
]

# Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools_list,
    input=history    
)


# Save function call outputs for subsequent requests
history += response.output


for item in response.output:
    if item.type == "function_call":
        if item.name == "weather_in_city":
            
            weather_str = weather_in_city(**json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            history.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "weather information": weather_str
                })
            })


print("Final input:")
print(history)

# Now tell the LLM about the result of the function call
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with the weather information generated by a tool.",
    tools=tools_list,
    input=history,
)

Final input:
[{'role': 'user', 'content': 'What is the current weather in Kempten?'}, ResponseReasoningItem(id='rs_00278c6cdeda3bc4006901efffaa00819192030d356d22a62b', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"city":"Kempten"}', call_id='call_X1m7G3TsLEEpzkvPA63NLrI1', name='weather_in_city', type='function_call', id='fc_00278c6cdeda3bc4006901f006bb3c8191ad0f74b5f1f2235f', status='completed'), {'type': 'function_call_output', 'call_id': 'call_X1m7G3TsLEEpzkvPA63NLrI1', 'output': '{"weather information": "The weather in Kempten is sunny and warm (45\\u00b0F)"}'}]


In [66]:
response.model_dump()

{'id': 'resp_00278c6cdeda3bc4006901f00767e881919059f2cad8eb9a55',
 'created_at': 1761734663.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'Respond only with the weather information generated by a tool.',
 'metadata': {},
 'model': 'gpt-5-2025-08-07',
 'object': 'response',
 'output': [{'id': 'rs_00278c6cdeda3bc4006901f00832e48191be72e61ac6e5d3de',
   'summary': [],
   'type': 'reasoning',
   'content': None,
   'encrypted_content': None,
   'status': None},
  {'id': 'msg_00278c6cdeda3bc4006901f00db6d081919d2dfe35c2aaa4d8',
   'content': [{'annotations': [],
     'text': 'The weather in Kempten is sunny and warm (45°F)',
     'type': 'output_text',
     'logprobs': []}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [{'name': 'weather_in_city',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'The name o

In [67]:
response.output_text

'The weather in Kempten is sunny and warm (45°F)'

# LLMs and web search

In [3]:
import os
from openai import OpenAI


f = open("my_secret_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()
client = OpenAI(api_key=my_secrete_key)

prompt = """
            Was für Informatik-Masterstudiengänge hat die Hochschule Kempten aktuell im Wintersemester 2025/2026 anzubieten?
            Bitte gebe mir die Antwort exakt im Folgenden Outputformat zurück OHNE weitere Zusatzinfos!
            - <Name des Studienfaches> / <Anzahl der Semester> / <Ist Start im WS möglich? 1/0> / <Ist Start im SS möglich? 1/0>
            - <Name des Studienfaches> / <Anzahl der Semester> / <Ist Start im WS möglich? 1/0> / <Ist Start im SS möglich? 1/0>
            - ...
         """

response = client.responses.create(
    model="gpt-5",
    tools=[{"type": "web_search"}],
    instructions="",
    input=prompt,
)


print(response.output_text)

- Game Engineering und Visual Computing / 3 / 1 / 1
- Informatik / 3 / 1 / 1
- Künstliche Intelligenz und Computer Vision / 3 / 1 / 1


In [4]:
type(response.output_text)

str

In [6]:
response.output_text.split("\n")

['- Game Engineering und Visual Computing / 3 / 1 / 1',
 '- Informatik / 3 / 1 / 1',
 '- Künstliche Intelligenz und Computer Vision / 3 / 1 / 1']

In [8]:
import os
from openai import OpenAI


f = open("my_secret_api_key.txt", "rt")
my_secrete_key = f.read()
f.close()
client = OpenAI(api_key=my_secrete_key)





anfrage1 = """
            Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision?
          """

anfrage2 = """
            Ohh!! So viel zu tun! Brauche mehr Zeit bis zur Abgabe meiner Masterarbeit!
            Können Sie die Frist bitte einen Monat nach hinten schieben!
          """

anfrage3 = """
            Wir sind die Firma SuperSki3000. Wir suchen Studenten als Skilehrer.
            Dürften wir am Eingang hierzu ein Poster bei Ihnen aufhängen?
           """

anfrage4 = """
            Darf ich das Busticket eigentlich auch in den Semesterferien nutzen?
           """

anfrage5 = """
            Was gibt es heute in der Mensa zu essen?
           """

anfrage6 = """
            Darf ich mein Auto auch am Wochenende in der Tiefgarage parken?
           """

anfragen = [anfrage1,
            anfrage2,
            anfrage3,
            anfrage4,
            anfrage5,
            anfrage6 ]

for anfrage in anfragen:

  prompt = f"""
            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.

            Hier die Anfrage:
            ---
            {anfrage}
            ---

            Du sollst jede Anfrage klassifizieren in eine der folgenden 5 Klassen:
            ---
            1. Anfrage zu einem speziellem Studium
            2. Antrag auf Fristverlängerung für Abschlussarbeit
            3. Anfrage wegen Veröffentlichung Stellenangebote
            4. Anfrage wegen Semesterticket
            5. Anfrage Sonstiges
            ---
            Antworte nur mit 1-5
          """
  
  print(prompt)

  response = client.responses.create(
      model="gpt-5",    
      instructions="",
      input=prompt,
  )

  print(f"Das LLM klassifiziert diese Anfrage als: {response.output_text}")



            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.

            Hier die Anfrage:
            ---
            
            Wieviele Semester dauert das Studium Künstliche Intelligenz und Computer Vision?
          
            ---

            Du sollst jede Anfrage klassifizieren in eine der folgenden 5 Klassen:
            ---
            1. Anfrage zu einem speziellem Studium
            2. Antrag auf Fristverlängerung für Abschlussarbeit
            3. Anfrage wegen Veröffentlichung Stellenangebote
            4. Anfrage wegen Semesterticket
            5. Anfrage Sonstiges
            ---
            Antworte nur mit 1-5
          
Das LLM klassifiziert diese Anfrage als: 1

            Du bist ein Assistent zur Klassifikation von Benutzeranfragen auf der
            Webseite der Hochschule Kempten.

            Hier die Anfrage:
            ---
            
            Ohh!! So viel zu tun! Brauche mehr 

In [9]:
response.output_text

'5'